In [ ]:
import os, sys
sys.path.append(os.path.abspath('../src/bin/lib/')) # add path to lib

In [2]:
import yaml
import json
from yaml import CLoader as Loader, CDumper as Dumper
import data_operations as dop
from dotted_dict import DottedDict
from collections import namedtuple
import nmdc
import data_operations as dop
from pandasql import sqldf
from pprint import pprint
import pandas as pds
def pysqldf(q):
    return sqldf(q, globals())

In [3]:
spec_file = "../src/bin/lib/nmdc_data_source.yaml"
with open(spec_file, 'r') as input_file:
    spec = DottedDict(yaml.load(input_file, Loader=Loader))

In [4]:
# mdf = dop.make_dataframe_from_spec_file (spec_file, nrows=20) #  for testing grab 20 rows
mdf = dop.make_dataframe_from_spec_file (spec_file)
# mdf.head()

## Test building study json

In [5]:
study_table = dop.extract_table(mdf, 'study_table')
contact_table = dop.extract_table(mdf, 'contact_table')
proposals_table = dop.extract_table(mdf, 'proposals_table')
project_table = dop.extract_table(mdf, 'project_table')
project_biosample_table = dop.extract_table(mdf, 'project_biosample_table')
biosample_table = dop.extract_table(mdf, 'biosample_table')

In [6]:
study = dop.make_study_dataframe(study_table, contact_table, proposals_table)
study_dictdf = study.to_dict(orient="records") # transorm dataframe to dictionary

In [7]:
## specify attributes
attributes = \
    ['gold_study_name', 'principal_investigator_name', 'add_date', 'mod_date', 'doi',
      'ecosystem', 'ecosystem_category', 'ecosystem_type', 'ecosystem_subtype', 'specific_ecosystem', 'ecosystem_path_id']

## create list of json string objects
study_json_list = dop.make_json_string_list\
    (study_dictdf, nmdc.Study, id_key='gold_id', name_key='study_name', description_key="description", attribute_fields=attributes)

In [8]:
# print(json.dumps(json.loads(study_json_list[0]), indent=4)) ## peek at data

## Test building project json

In [9]:
project = dop.make_project_dataframe(project_table, study_table, contact_table)
project_dictdf = project.to_dict(orient="records") # transorm dataframe to dictionary

In [10]:
## specify characteristics
attributes = \
    ['add_date', 'mod_date', 'completion_date', 'ncbi_project_name', 'omics_type', 'principal_investigator_name', 'processing_institution']

## create list of json string objects
project_json_list = dop.make_json_string_list\
    (project_dictdf, nmdc.OmicsProcessing, id_key='gold_id', name_key='project_name', 
     part_of_key="study_gold_id", description_key="description", attribute_fields=attributes)

In [11]:
# print(json.dumps(json.loads(project_json_list[0]), indent=4)) ## peek at data

## Test building biosample json

In [12]:
biosample = dop.make_biosample_dataframe(biosample_table, project_biosample_table, project_table)
biosample['lat_lon'] = biosample.apply(lambda row: dop.make_lat_lon(row.latitude, row.longitude), axis=1)
biosample.lat_lon
# biosample_dictdf[0] ## peek at data

AttributeError: module 'data_operations' has no attribute 'make_lat_lon'

In [ ]:
## specify attributes
attributes = \
    ['lat_lon',
     'add_date',
     'mod_date',
     'ecosystem_path_id',
     'ecosystem',
     'ecosystem_category',
     'ecosystem_type',
     'ecosystem_subtype',
     'specific_ecosystem',
     'habitat',
     'location',
     'community',
     'ncbi_taxonomy_name',
     'geographic_location',
     'latitude',
     'longitude',
     'sample_collection_site',
     'identifier',
     'sample_collection_year',
     'sample_collection_month',
     'sample_collection_day',
     'sample_collection_hour',
     'sample_collection_minute',
     'host_name',
     'depth',
     'subsurface_depth',
     'altitude',
     'temperature_range',
     'proport_woa_temperature',
     'biogas_temperature',
     'growth_temperature',
     'soil_annual_season_temp',
     'water_samp_store_temp',
     'biogas_retention_time',
     'salinity',
     'pressure',
     'ph',
     'chlorophyll_concentration',
     'nitrate_concentration',
     'oxygen_concentration',
     'salinity_concentration'
    ]

In [ ]:
# len(biosample)

In [ ]:
## create list of json string objects
biosample_json_list = dop.make_json_string_list\
    (biosample_dictdf, nmdc.Biosample, id_key='gold_id', name_key='biosample_name', 
     part_of_key="project_gold_ids", description_key="description", attribute_fields=attributes)

In [ ]:
# print(json.dumps(json.loads(biosample_json_list[0]), indent=4)) ## peek at data

## Test building EMSL data